## **Computer vision project: Deggendorf Waste Sorting Assistant**

### **Overview**
The Deggendorf Waste Sorting Assistant is a Computer Vision-based tool designed to help residents and international students correctly identify waste bins. The project leverages image classification to determine the category of a given waste bin based on its visual characteristics. Users can take a picture of an unlabeled bin, and the model will classify it while providing information on the appropriate waste materials for disposal.

### **Project Goals**
- Develop an image classification model capable of identifying waste bins in Deggendorf.
- Provide users with clear guidance on proper waste disposal based on bin classification.
- Document all processes in a Jupyter Notebook, covering dataset creation, model training, evaluation, and deployment.


### **Dataset Creation**
- Collect and photograph waste bins from different locations in Deggendorf, including Restmüll, Bioabfall, Gelber Sack, and Papier.
- Label images according to the waste bin category and permissible waste items.
- Preprocess images by applying normalization, augmentation, and resizing techniques.

### **Model Training**
- Implement a Convolutional Neural Network (CNN) for waste bin classification.
- Train the model using the labeled dataset.
- Evaluate the model's performance using accuracy, precision, recall, and confusion matrices.

### **Technologies Used**
- **Python** for implementation.
- **OpenCV** for image processing.
- **TensorFlow/Keras or PyTorch** for model development.
- **Jupyter Notebook** for interactive documentation.

### **Expected Outcome**
- A trained image classification model for waste bin recognition.
- A fully documented Jupyter Notebook covering all project aspects.
- A functional prototype that can classify waste bins and provide disposal instructions.

### **Future Enhancements**
- Expansion to other cities with diverse waste bin colors and categories.
- Real-time bin identification through video streams.
- Multilingual support for a broader audience.
- GPS-based recommendations for nearby waste disposal sites.

This project contributes to sustainability efforts by promoting proper recycling practices and reducing waste contamination through an easy-to-use, technology-driven solution.
